**TM10007 Machine Learning**

In [1]:
!git clone https://github.com/jveenland/tm10007_ml.git

import zipfile
import os
import pandas as pd

with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

Cloning into 'tm10007_ml'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 83 (delta 13), reused 12 (delta 12), pack-reused 64
Unpacking objects: 100% (83/83), 67.93 MiB | 4.56 MiB/s, done.
The number of samples: 827
The number of columns: 9001


In [2]:
# import function
from sklearn import model_selection


In [5]:
data['label'].sum()

146

In [11]:
## test test test test


In [7]:
ECG_NON_NORMAL

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,...,11_741,11_742,11_743,11_744,11_745,11_746,11_747,11_748,11_749,label
1,152.726718,297.353726,111.168880,25.419423,14.636085,24.609688,49.533842,253.651571,483.852043,167.617409,...,0.804160,0.809705,0.830032,0.774368,0.798178,0.853519,0.827946,0.859936,0.742673,1
12,6.323226,15.953172,26.120406,5.109627,4.012020,16.225039,114.460883,8.289232,4.147352,5.958195,...,0.110629,0.164965,0.081171,0.026519,0.105995,0.138955,0.004136,0.064466,0.039308,1
15,18.644972,24.808661,29.530931,34.786282,81.380679,91.139503,268.234344,99.557117,143.066045,113.213748,...,1.330045,1.347135,1.230999,1.297284,1.304806,1.270956,1.294159,1.234423,1.282566,1
18,247.728859,82.530058,64.737390,101.079961,313.343333,144.387597,52.225949,43.735204,159.002631,433.966373,...,0.521610,0.590781,0.606558,0.529782,0.583035,0.523189,0.579762,0.534765,0.551470,1
23,7.554942,18.251545,19.131640,4.189196,13.567905,13.140102,2.874134,6.775483,5.447489,62.872336,...,0.122462,0.077219,0.211491,0.115538,0.179688,0.105979,0.159539,0.067537,0.086839,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,883.944360,198.173280,60.302690,15.945958,11.504500,9.191698,4.707053,5.127655,9.140042,22.027639,...,0.040732,0.092464,0.184638,0.110685,0.124618,0.122629,0.197854,0.256129,0.099834,1
787,875.085192,77.927478,41.706337,21.197205,11.304032,5.111424,146.548466,11.372778,5.841510,3.017511,...,0.203799,0.156882,0.143697,0.094681,0.175585,0.198802,0.200527,0.055031,0.086733,1
789,823.013130,369.643146,168.177460,57.124367,41.706590,19.712585,8.300908,12.194639,3.027491,15.672809,...,1.524782,1.493644,1.396806,1.371257,1.399370,1.401800,1.417314,1.457833,1.471118,1
790,278.810394,206.533025,68.386743,68.101193,12.766478,106.676858,57.423892,41.718276,50.814634,79.378399,...,0.147504,0.155291,0.157302,0.119661,0.070990,0.183585,0.097615,0.201147,0.084877,1


In [ ]:





x_train, x_test = model_selection.train_test_split(data, test_size=0.2)
#print(x_train)





print(f'The number of samples: {len(x_train.index)}')
print(f'The number of columns: {len(x_train.columns)}')
# Stratified split met vergelijkbare ratio 


NameError: ignored